In [ ]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

# Ruta de la carpeta donde se encuentran las imágenes
ruta_imagenes = '/content/drive/MyDrive/Colab Notebooks/PROYECTO/AllClases/'

# Lista para almacenar las imágenes
imagenes = []

# Lista para almacenar las etiquetas
etiquetas = []

# Obtener una lista de todos los archivos en la carpeta
archivos = os.listdir(ruta_imagenes)

def umbralizar(imagen):
    # Verificar si la imagen se leyó correctamente
    if imagen is None:
        print("No se pudo leer la imagen.")
        return None

    # Umbralizar y luego invertir los valores de los píxeles
    imagen_binarizada = np.where(imagen > 127, 255, 0).astype(np.uint8)
    return imagen_binarizada

# Recorrer todos los archivos
for archivo in archivos:
  # Verificar si el archivo es una imagen
  if archivo.endswith(".JPG"):
    # Leer la imagen en escala de grises
    imagen = cv2.imread(os.path.join(ruta_imagenes, archivo),0)
     # Obtener la etiqueta de la imagen
    etiqueta = archivo.split("_")[0]

    # Agregar la imagen y la etiqueta a las listas correspondientes
    imagenes.append(umbralizar(imagen))
    etiquetas.append(etiqueta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Función para extraer descriptores SIFT de una imagen
def extract_sift_descriptors(image):
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(image, None)
    return descriptors

# Supongamos que tienes un array 'images' con las imágenes y un array 'labels' con las etiquetas
# Asegúrate de que las imágenes tengan el mismo tamaño

# Crear listas para almacenar los descriptores y las etiquetas de cada imagen
descriptors_list = []
labels_list = []

# Extraer los descriptores SIFT y las etiquetas de cada imagen
for image, label in zip(imagenes, etiquetas):
    sift_descriptors = extract_sift_descriptors(image)
    descriptors_list.extend(sift_descriptors)
    labels_list.extend([label] * len(sift_descriptors))

# Convertir las listas a arrays de NumPy
descriptors = np.vstack(descriptors_list)
labels = np.array(labels_list)

# Convertir etiquetas de texto a etiquetas numéricas
label_encoder = LabelEncoder()
labels_numeric = label_encoder.fit_transform(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(descriptors, labels_numeric, test_size=0.2, random_state=42)

# Crear un modelo k-NN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = knn_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5721582866637946
